In [1]:
import pytrends
import pandas as pd
import lxml
import requests
# import numpy as np
#Creamos un DataFrame vacío.
df = pd.DataFrame()


In [2]:
#TrendReq recibe dos parámetros importantes.
#-hl significa lenguaje de alojamiento para acceder a Google Trends; en este ejemplo, configuramos English.
# -tz significa zona horaria, en este ejemplo, usamos la zona horaria de Europa central, dónde se encuentra España. (representada en minutos ), que es 60.
#https://forbrains.co.uk/international_tools/earth_timezones
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=60)

In [3]:
import sqlite3
# Conexión con la bbdd
conn = sqlite3.connect('noticiascriptomonedas.db')
c = conn.cursor()


#list = ['bitcoin','btc','mana', 'mana crypto','decentraland','cryptocurrency','crypto','metaverse','nft','nfts'] # list of keywords to get data
# list = [['bitcoin'],['btc']]
list=['bitcoin']



for topic in list:
       # topic=topic[0]
       print('Se está buscando:',topic)
       list_topics_bbdd=c.execute('SELECT criterio FROM TOPIC_G').fetchall()
       list_topics_bbdd=[item[0] for item in list_topics_bbdd]

       if topic not in list_topics_bbdd:
              try:
                     c.execute('INSERT INTO TOPIC_G(criterio) VALUES (?)',(topic,))
                     conn.commit()
                     print('Se ha añadido a TOPIC_G:',topic)
              except sqlite3.Error as error:
                     print("No se ha podido añadir (?) en la tabla TOPIC_G:", (topic,error))

       # kw_list=Criterio
       # kw_list=['bitcoin','btc']

       # El estudio se ha realizado teniendo en cuenta las interaciones de las redes sociales comprendidas entre el 1 de marzo- 17 de abril
       # pytrends.build_payload(kw_list, cat=0, timeframe=('2022-03-01 2022-04-17'))
       # data = pytrends.interest_over_time()
       # data = data.reset_index()
       # df = data[['date','bitcoin','btc']]
       data= pytrends.get_historical_interest(topic, year_start=2022, month_start=3, day_start=1, hour_start=0, year_end=2022, month_end=4, day_end=17, hour_end=0, cat=0, geo='', gprop='', sleep=60)
       print(data)
       df = pd.DataFrame(data)
       df = df.reset_index()
       date = df['date']
       search = df[topic]  
       for i in range(len(search)):
              c.execute("""INSERT INTO PRINCIPAL_G(fecha, busquedas) VALUES (?,?);""",(date[i],int(search[i]),))
              c.execute("""INSERT INTO INCLUIDO_G VALUES ((SELECT id_topicg FROM TOPIC_G WHERE criterio=(?)),(SELECT id_principalg FROM PRINCIPAL_G ORDER BY id_principalg DESC LIMIT 1 ));""",(topic,))

# conn.commit()
       
c.close()
conn.close()

Se está buscando: bitcoin
The request failed: Google returned a response with code 400.
